In [100]:
from typing import *
import pandas as pd
import libcst as cst
import re

In [101]:
example_code = open('example.py', 'r').read()
print(example_code)

class A:
    def __init__(self):
        pass

    def _a(self):
        pass

    def b(self):
        pass

    B = 0



In [102]:
example_code_cst = cst.parse_module(example_code)
example_code_cst

Module(
    body=[
        ClassDef(
            name=Name(
                value='A',
                lpar=[],
                rpar=[],
            ),
            body=IndentedBlock(
                body=[
                    FunctionDef(
                        name=Name(
                            value='__init__',
                            lpar=[],
                            rpar=[],
                        ),
                        params=Parameters(
                            params=[
                                Param(
                                    name=Name(
                                        value='self',
                                        lpar=[],
                                        rpar=[],
                                    ),
                                    annotation=None,
                                    equal=MaybeSentinel.DEFAULT,
                                    default=None,
                                    comma=MaybeSentin

In [97]:
class_def = example_code_cst.body[0]
# example_code_cst.body[0] = class_def.with_changes(whitespace_before_colon=cst.SimpleWhitespace(value=' '))

In [98]:
class_def.body.header, class_def.body.footer, class_def.body.indent

(TrailingWhitespace(
     whitespace=SimpleWhitespace(
         value='',
     ),
     comment=None,
     newline=Newline(
         value=None,
     ),
 ),
 [],
 None)

In [66]:
print(example_code_cst.code)

class A\
 :
    def __init__(self):
        pass

    def _a(self):
        pass

    def b(self):
        pass

    B = 0



In [67]:
example_code_cst.body[0].body

IndentedBlock(
    body=[
        FunctionDef(
            name=Name(
                value='__init__',
                lpar=[],
                rpar=[],
            ),
            params=Parameters(
                params=[
                    Param(
                        name=Name(
                            value='self',
                            lpar=[],
                            rpar=[],
                        ),
                        annotation=None,
                        equal=MaybeSentinel.DEFAULT,
                        default=None,
                        comma=MaybeSentinel.DEFAULT,
                        star='',
                        whitespace_after_star=SimpleWhitespace(
                            value='',
                        ),
                        whitespace_after_param=SimpleWhitespace(
                            value='',
                        ),
                    ),
                ],
                star_arg=MaybeSentinel.DEFAULT,
   

In [5]:
def sort_functions(functions_list: list):
    df = pd.DataFrame([(b.name.value, b) for b in functions_list], columns=['func_name', 'body'])
    df['is_magic'] = df['func_name'].str.startswith('__') & df['func_name'].str.endswith('__')
    df['is_public'] = ~df['func_name'].apply(lambda x: re.search('^_[a-zA-Z]', x) is not None)
    df_2 = df.sort_values(['is_magic', 'is_public'], ascending=False)
    sorted_functions_list = df_2['body'].tolist()
    return sorted_functions_list

In [6]:
class TypingTransformer(cst.CSTTransformer):
    def visit_ClassDef(self, node: cst.IndentedBlock) -> Optional[bool]:
        pass
        
    def leave_ClassDef(self, original_node: cst.ClassDef, updated_node: cst.ClassDef) -> cst.CSTNode:        
        if isinstance(original_node, cst.ClassDef):
            print('changing')
            new_body = updated_node.body.with_changes(body=sort_functions(updated_node.body.body))
            updated_node_2 = updated_node.with_changes(body=new_body)
            return updated_node_2
        return updated_node

In [7]:
transformer = TypingTransformer()
modified_tree = example_code_cst.visit(transformer)
print(example_code_cst.code)
print(modified_tree.code)

changing
class A:
    def __init__(self):
        pass

    def _a(self):
        pass

    def b(self):
        pass

class A:
    def __init__(self):
        pass

    def b(self):
        pass

    def _a(self):
        pass



In [6]:
example_code_cst.body[0].body

IndentedBlock(
    body=[
        FunctionDef(
            name=Name(
                value='__init__',
                lpar=[],
                rpar=[],
            ),
            params=Parameters(
                params=[
                    Param(
                        name=Name(
                            value='self',
                            lpar=[],
                            rpar=[],
                        ),
                        annotation=None,
                        equal=MaybeSentinel.DEFAULT,
                        default=None,
                        comma=MaybeSentinel.DEFAULT,
                        star='',
                        whitespace_after_star=SimpleWhitespace(
                            value='',
                        ),
                        whitespace_after_param=SimpleWhitespace(
                            value='',
                        ),
                    ),
                ],
                star_arg=MaybeSentinel.DEFAULT,
   